In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from Mylib import *

In [ ]:
def gen_dateRanges(s, e, by = 1):
    start = datetime.date(s[0], s[1], s[2])
    end = datetime.date(e[0], e[1], e[2])
    numdays = end - start
    date_list = [(end - datetime.timedelta(days=x)).strftime("%Y-%m-%d")
                 for x in range(numdays.days + 1)][::-1]
    return [[date_list[i:i + by][idx] for idx in [0, -1]]
            for i in range(0, len(date_list), by) if len(date_list[i:i + by]) == by]

### References for Request Body
https://developers.google.com/analytics/devguides/reporting/core/v4/rest/v4/reports/batchGet#ReportRequest

In [ ]:
my_API = GoogleAPI()
ar = my_API.initialize_serivce('./float-222704-70edd21a1124.json', 
                               ['https://www.googleapis.com/auth/analytics.readonly'], 
                               'analyticsreporting', 'v4')

gs = my_API.initialize_serivce('./gscred.json', 
                               ['https://www.googleapis.com/auth/spreadsheets'],
                               'sheets', 'v4')

In [ ]:
METRICS = [
    'sessionDuration', 'bounces', 'sessions'
]

In [ ]:
GN = buildDF(gen_dateRanges([2020, 4, 1], [2020, 4, 30], 1), [
    'date', 'sessionCount', 
    'socialNetwork',
    'country', 'city' # Geo Network
    ], METRICS)

In [ ]:
AD = buildDF(gen_dateRanges([2020, 4, 1], [2020, 4, 30], 30), [
    'date', 'sessionCount', 
    'socialNetwork',
    'interestAffinityCategory', 'userAgeBracket', 'userGender', # Audience
    ], METRICS, ['United States'])
# AD.interestAffinityCategory = AD.interestAffinityCategory.apply(lambda x: re.sub('/.+', '', x))

In [ ]:
GN.describe(include='all')

In [ ]:
AD.describe(include='all')

In [ ]:
GN.to_csv('./Data_GeoNetwork.csv')
AD.to_csv('./Data_Audience.csv')

## Get Result Tables as Google Sheet

In [ ]:
files = ['session_distribution_by_city', 'session_distribution_by_aff',
         'session_distribution_by_age', 'session_distribution_by_sex']

tables = [pd.read_csv(f'./Analysis/{file}.csv') for file in files]
MakeSheet(gs, {f'{n}': table for n, table in zip(files, tables)}, 'session_distributions')